In [21]:
# HIDE
import pandas as pd
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True, context="talk")
from IPython import display
%matplotlib inline

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import keras as ke
import keras.backend as K
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import load_model


create_gif = False

# model reconstruction from JSON:
from keras.models import model_from_json

print(f"sklearn: {sk.__version__}")
print(f"pandas: {pd.__version__}")
print(f"kerads: {ke.__version__}")


import src.my_functions_product1 as myFC
import matplotlib.pyplot as plt


sklearn: 0.19.1
pandas: 0.23.0
kerads: 2.1.6


In [22]:
"""# reading the file from a path and list the field names and input columns of interest

#path = 'data/adult/adult.data'
#output_path = 'Feature-Datasets/adult/'
#arc_id = 'user1'


path = 'data/bank/bank.csv'
output_path = 'Feature-Datasets/bank/'
arc_id = 'user1'

X_df, Ybin, Zbin, result_fname = myFC.read_process_data_output_bias(path)"""

"# reading the file from a path and list the field names and input columns of interest\n\n#path = 'data/adult/adult.data'\n#output_path = 'Feature-Datasets/adult/'\n#arc_id = 'user1'\n\n\npath = 'data/bank/bank.csv'\noutput_path = 'Feature-Datasets/bank/'\narc_id = 'user1'\n\nX_df, Ybin, Zbin, result_fname = myFC.read_process_data_output_bias(path)"

In [23]:
# reading the file from a path and list the field names and input columns of interest

input_option = 'not manual' #'manual' or 'not manual'

#--------------------------------------
if False:
    
    path = 'Data/adult/'
    data_filename = 'adult.data'
    cprediction_field_exists = 'N'
    cprediction_field = ['N']
    target_field = ' <=50K'
    target_label0 = ' <=50K'
    target_label1 = ' >50K'
    sensitive_field = ' Male'
    not_to_consider_fields = [' Male']
    sensitive_class0 = ' Male'
    sensitive_class1 = ' Female'
    good_outcome = 'Income >50K'
    output_path = 'Feature-Datasets/bank/'


#--------------------------------------
if True:

    path = 'Data/bank/'
    data_filename = 'bank.csv'
    cprediction_field_exists = 'N'
    cprediction_field = ['N']
    target_field = 'y'
    target_label0 = 'no'
    target_label1 = 'yes'
    sensitive_field = 'age'
    not_to_consider_fields = ['age']
    sensitive_class0 = '<25 or >60'
    sensitive_class1 = '25-60'
    good_outcome = 'Subsribed a deposit'
    output_path = 'Feature-Datasets/bank/'


#--------------------------------------


X_df, Ybin, Zbin, result_fname = myFC.read_process_data_output_bias(path+data_filename, input_option,cprediction_field_exists,cprediction_field,target_field, sensitive_field, not_to_consider_fields,sensitive_class0,sensitive_class1,target_label0,target_label1)   



Columns in your data are:
  ['age' 'job' 'marital' 'education' 'default' 'housing' 'loan' 'contact'
 'month' 'day_of_week' 'duration' 'campaign' 'pdays' 'previous' 'poutcome'
 'emp.var.rate' 'cons.price.idx' 'cons.conf.idx' 'euribor3m' 'nr.employed'
 'y']
burasi1
age
age
burasi2
target field:y
target field:y Label 0-1 no, yes
sensitive field:age Class 0-1 <25 or >60, 25-60
not to consider fields ['age']
result file:  Data--bank--bank.csv_CP_N_T_y_no_yes_S_age_<25_or_>60_25-60_
Target field: p_rule_for_Y1 29.89


### Feature Creation

In [24]:
X_train, X_test, y_train, y_test, Z_train, Z_test = myFC.feature_creation(X_df, Ybin, Zbin)

In [25]:
feature_train_df = X_train.copy()
feature_train_df['y'] = y_train
feature_train_df['Z'] = Z_train
feature_train_df.to_csv(output_path+'train'+'.csv', index=False)

feature_test_df = X_test.copy()
feature_test_df['y'] = y_test
feature_test_df['Z'] = Z_test
feature_test_df.to_csv(output_path+'test'+'.csv', index=False)




In [26]:
X_train.head(5)

,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_blue-collar,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
36449,-0.719636,-0.202049,0.194542,1.640574,-1.896503,-1.057758,-0.051736,-1.356397,-1.256373,-0.537342,...,-0.704739,-0.334824,-0.130913,-0.115691,-0.511523,-0.516882,2.026106,-0.497434,-2.519871,-0.183944
33073,-0.362081,-0.202049,0.194542,-0.346019,-1.195943,-1.178759,-1.222170,-1.339684,-0.939519,-0.537342,...,1.418965,-0.334824,-0.130913,-0.115691,-0.511523,-0.516882,2.026106,-0.497434,0.396846,-0.183944
11656,-0.505103,4.202676,0.194542,-0.346019,0.842048,1.538577,-0.268483,0.774274,0.845379,-0.537342,...,-0.704739,-0.334824,-0.130913,-0.115691,-0.511523,-0.516882,-0.493558,-0.497434,0.396846,-0.183944
5510,-0.395955,-0.569109,0.194542,-0.346019,0.650987,0.724413,0.901952,0.715489,0.332048,-0.537342,...,1.418965,-0.334824,-0.130913,-0.115691,1.954948,-0.516882,-0.493558,-0.497434,0.396846,-0.183944
37348,-0.215296,-0.569109,0.194542,-0.346019,-1.896503,-2.374941,1.985688,-1.574824,-1.256373,-0.537342,...,-0.704739,-0.334824,-0.130913,-0.115691,-0.511523,-0.516882,2.026106,-0.497434,0.396846,-0.183944


### Setting up architectures

In [27]:
#==============================================================================
def User_Classifier_arch(n_features):
    inputs = Input(shape=(n_features,))
    dense1 = Dense(32, activation='relu')(inputs)
    dropout1 = Dropout(0.2)(dense1)
    dense2 = Dense(32, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)
    dense3 = Dense(32, activation="relu")(dropout2)
    dropout3 = Dropout(0.2)(dense3)
    dense4 = Dense(32, activation="relu")(dropout3)
    dropout4 = Dropout(0.2)(dense4)
    outputs = Dense(1, activation='sigmoid')(dropout4)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model    
#==============================================================================
def User_Adversary_arch(inputs):
    dense1 = Dense(32, activation='relu')(inputs)
    dense2 = Dense(32, activation='relu')(dense1)
    dense3 = Dense(32, activation='relu')(dense2)
    dense4 = Dense(32, activation="relu")(dense3)
    outputs = Dense(1, activation='sigmoid')(dense4)
    return Model(inputs=[inputs], outputs=outputs)   
#==============================================================================
def User_main_task_adv_architecture(n_features):
    main_task_arch = User_Classifier_arch(n_features)
    main_task_arch_json_string = main_task_arch.to_json()
    adv_inputs = Input(shape=(1,))
    adv_task_arch = User_Adversary_arch(adv_inputs)
    adv_task_arch_json_string = adv_task_arch.to_json()
    return main_task_arch_json_string, adv_task_arch_json_string  

## Saving architectures

In [28]:
arc_id = 'user1'

n_features = X_train.shape[1]
main_task_arch_json_string, adv_task_arch_json_string = User_main_task_adv_architecture(n_features)

main_task_arch_json_string_file = output_path+arc_id+'_main_task_arch_json_string_file'+'.txt'
with open(main_task_arch_json_string_file,"wt") as out_file:
    out_file.write(main_task_arch_json_string)
    
adv_task_arch_json_string_file = output_path+arc_id+'_adv_task_arch_json_string_file'+'.txt'
with open(adv_task_arch_json_string_file,"wt") as out_file:
    out_file.write(adv_task_arch_json_string)    

## Model and compile only main task: check prediction results

In [29]:
h5_file_name = output_path+arc_id+'_main_task_ori_trained_model.h5'
save_the_weights = True

main_task_accuracy, p_rule_for_Y1, y_pred = myFC.pre_train_main_task(main_task_arch_json_string, X_train, y_train,X_test,y_test,Z_test, save_the_weights=save_the_weights, h5_file_name= h5_file_name)

Accuracy: 91.29


In [30]:
p_rule_for_Y1


'27.17'